In [175]:
import folium
import pandas as pd
from scipy.spatial import Voronoi
import branca.colormap as cm
import numpy as np

estaciones = pd.read_csv('https://datos.madrid.es/egob/catalogo/211346-1-estaciones-acusticas.csv', sep=';', parse_dates=['FECHA ALTA'])
ruido = pd.read_csv('https://datos.madrid.es/egob/catalogo/215885-10749127-contaminacion-ruido.csv', sep=';', decimal=',')

In [176]:
estaciones.head()

,NMT,NOMBRE,DIRECCION,LONGITUD,LATITUD,X (ETRS89),Y (ETRS89),ALTITUD,FECHA ALTA
0,RF 1,Pº Recoletos,Frente al Nº 23 del Paseo de Recoletos,-3.691877,40.422599,441307,4474893,648,07/03/2011
1,RF 2,Carlos V,"Plaza del Emperador Carlos V, junto al Nº 1 de...",-3.691509,40.409121,441327,4473397,629,01/12/1998
2,RF 3,Plaza del Carmen,Plaza del Carmen frente al Nº 3 de la Calle de...,-3.703175,40.419251,440346,4474529,657,01/11/1999
3,RF 4,Plaza de España,Frente al Nº 18 de la Plaza de España,-3.712253,40.424005,439580,4475063,637,01/12/1998
4,RF 5,Bº del Pilar,Parque de La Vaguada semiesquina de Avenida Mo...,-3.711543,40.478197,439688,4481078,673,07/06/1999


In [177]:
estaciones.dtypes

NMT            object
NOMBRE         object
DIRECCION      object
LONGITUD      float64
LATITUD       float64
X (ETRS89)      int64
Y (ETRS89)      int64
ALTITUD         int64
FECHA ALTA     object
dtype: object

In [178]:
estaciones['NMT'] = estaciones['NMT'].str[3:].astype(int)

In [179]:
estaciones.head()

,NMT,NOMBRE,DIRECCION,LONGITUD,LATITUD,X (ETRS89),Y (ETRS89),ALTITUD,FECHA ALTA
0,1,Pº Recoletos,Frente al Nº 23 del Paseo de Recoletos,-3.691877,40.422599,441307,4474893,648,07/03/2011
1,2,Carlos V,"Plaza del Emperador Carlos V, junto al Nº 1 de...",-3.691509,40.409121,441327,4473397,629,01/12/1998
2,3,Plaza del Carmen,Plaza del Carmen frente al Nº 3 de la Calle de...,-3.703175,40.419251,440346,4474529,657,01/11/1999
3,4,Plaza de España,Frente al Nº 18 de la Plaza de España,-3.712253,40.424005,439580,4475063,637,01/12/1998
4,5,Bº del Pilar,Parque de La Vaguada semiesquina de Avenida Mo...,-3.711543,40.478197,439688,4481078,673,07/06/1999


In [54]:
estaciones.tail()

,NMT,Nombre,Dirección,X (ETRS89),Y (ETRS89),lat,lon
26,48,Castellana,"Jardín de Las Bellas Artes,a la altura del nº2...",441450,4476811,40.439887,-3.690369
27,50,Plaza de Castilla,Frente al nº 9 de la Plaza Castilla,441610,4479662,40.465581,-3.688744
28,54,Ensanche de Vallecas,Esquina entre la Avenida de la Gavia y la Aven...,448033,4469339,40.373011,-3.612142
29,55,Embajada II,Frente al nº 13 de la Calle Riaño,450779,4479239,40.462364,-3.580564
30,86,Tres Olivos,Frente al nº 23 de la Avenida del Campo de Cal...,441557,4483544,40.500548,-3.689727


In [9]:
ruido.head()

,NMT,anio,mes,dia,tipo,LAEQ,LAS01,LAS10,LAS50,LAS90,LAS99
0,3,2014,1,1,D,57.4,66.6,61.1,54.3,49.1,45.0
1,3,2014,1,1,E,58.7,66.6,62.0,56.0,52.1,48.5
2,3,2014,1,1,N,65.9,72.5,66.1,60.9,56.2,52.1
3,3,2014,1,1,T,62.2,69.3,63.8,56.5,50.3,45.8
4,4,2014,1,1,D,67.3,72.4,67.0,63.2,59.5,56.2


In [27]:
ruido['LAS01'].max()

106.3

In [171]:
coords = estaciones[['LATITUD', 'LONGITUD']].to_numpy()
min_lat = estaciones['LATITUD'].min() - 0.2 
max_lat = estaciones['LATITUD'].max() + 0.2
min_lon = estaciones['LONGITUD'].min() - 0.2
max_lon = estaciones['LONGITUD'].max() + 0.2

In [172]:
class VoronoiRegion:
    def __init__(self, region_id, station_id):
        self.id = region_id
        self.station_id = station_id
        self.vertices = []
        self.is_inf = False
        self.point_inside = None

    def __str__(self):
        text = f'region id={self.id}' + f'({self.station_id})'
        if self.point_inside:
            point_idx, point = self.point_inside
            text = f'{text}[point:{point}(point_id:{point_idx})]'
        text += ', vertices: '
        if self.is_inf:
            text += '(inf)'
        for v in self.vertices:
            text += f'{v}'
        return text

    def __repr__(self):
        return str(self)

    def add_vertex(self, vertex, vertices):
        if vertex == -1:
            self.is_inf = True
        else:
            point = vertices[vertex]
            self.vertices.append(point)
        
    @property
    def name(self):
        return estaciones[estaciones['NMT'] == self.station_id]['NOMBRE'].values[0]
    
    @property
    def coords(self):
        return self.point_inside[1]

def voronoi_to_voronoi_regions(voronoi, ids):
    voronoi_regions = []
    if len(ids) != len(voronoi.points):
        # Append None to the names list
        ids = np.append(ids, [None] * (len(voronoi.points) - len(ids)))

    for i, (point_region, station_id) in enumerate(zip(voronoi.point_region, ids)):
        region = voronoi.regions[point_region]
        vr = VoronoiRegion(point_region, station_id)
        for r in region:
            vr.add_vertex(r, voronoi.vertices)
        vr.point_inside = (i, voronoi.points[i])
        voronoi_regions.append(vr)
    return voronoi_regions


vor = Voronoi(np.append(coords, [[min_lat, min_lon], [min_lat, max_lon], [max_lat, min_lon], [max_lat, max_lon]], axis=0))
regions = voronoi_to_voronoi_regions(vor, estaciones['NMT'])
for r in regions:
    print(r)

region id=31(1)[point:[40.422599 -3.691877](point_id:0)], vertices: [40.43075631 -3.70043937][40.41574267 -3.69599029][40.41596666 -3.68778671][40.42969841 -3.68630657]
region id=32(2)[point:[40.409121 -3.691509](point_id:1)], vertices: [40.41574267 -3.69599029][40.40107985 -3.70872254][40.39686427 -3.70499418][40.40847812 -3.67757403][40.41596666 -3.68778671]
region id=34(3)[point:[40.419251 -3.703175](point_id:2)], vertices: [40.43075631 -3.70043937][40.43200519 -3.70227964][40.40548453 -3.71616807][40.40383089 -3.71475418][40.40107985 -3.70872254][40.41574267 -3.69599029]
region id=19(4)[point:[40.424005 -3.712253](point_id:3)], vertices: [40.43200519 -3.70227964][40.40548453 -3.71616807][40.41100081 -3.72441919][40.42061762 -3.72964388][40.44010819 -3.73220901][40.43321999 -3.70304503]
region id=7(5)[point:[40.478197 -3.711543](point_id:4)], vertices: [40.48298478 -3.69403358][40.51837785 -3.73023633][40.46846153 -3.76176043][40.45730683 -3.74342483][40.46242622 -3.70535502]
region

In [173]:
anio = 2024
mes = 3
dia = 1
tipo = 'N'
col = 'LAEQ'

In [174]:
m = folium.Map()
cmap = cm.LinearColormap(['green', 'yellow', 'red'], vmin=0, vmax=110)
for i, r in enumerate(regions):
    if r.is_inf:
        continue
    try:
        dBA = ruido[(ruido['NMT'] == r.station_id) & (ruido['anio'] == anio) & (ruido['mes'] == mes) & (ruido['dia'] == dia) & (ruido['tipo'] == tipo)][col].values[0]
        color = cmap(dBA)
    except:
        color = 'gray'
        dBA = -1
    folium.Polygon(locations=r.vertices, color=color, fill_color=color, fill_opacity=0.5, popup=dBA).add_to(m)
    folium.Marker(r.coords, icon=folium.Icon(color='blue'), popup=r.name).add_to(m)
m.fit_bounds(m.get_bounds())
m